Since there are many different dice calculation methods in Notebooks(a.k.a Kernels), I'm here to point out the right one.

In [ ]:
import torch

This competition is evaluated on the **mean Dice coefficient**. The Dice coefficient can be used to compare the pixel-wise agreement between a predicted segmentation and its corresponding ground truth. The formula is given by:

$Dice(X,Y) = \frac{2∗|X\cap{Y}|}{|X|+|Y|}$

where X is the predicted set of pixels and Y is the ground truth. The Dice coefficient is defined to be 1 when both X and Y are empty. The leaderboard score is the mean of the Dice coefficients for **each** *<ImageId, ClassId>* pair in the test set.

Which means the seperate channel of each mask will be average to Dice score. So we have 1801x4 samples in test set, when we evaluate this dice score on a batch, we should calculate dice as follows:

In [ ]:
def dice_channel_torch(probability, truth, threshold):
    batch_size = truth.shape[0]
    channel_num = truth.shape[1]
    mean_dice_channel = 0.
    with torch.no_grad():
        for i in range(batch_size):
            for j in range(channel_num):
                channel_dice = dice_single_channel(probability[i, j,:,:], truth[i, j, :, :], threshold)
                mean_dice_channel += channel_dice/(batch_size * channel_num)
    return mean_dice_channel


def dice_single_channel(probability, truth, threshold, eps = 1E-9):
    p = (probability.view(-1) > threshold).float()
    t = (truth.view(-1) > 0.5).float()
    dice = (2.0 * (p * t).sum() + eps)/ (p.sum() + t.sum() + eps)
    return dice

In [ ]:
batchsize = 16
channel_num = 4
probability = torch.rand(batchsize,channel_num,256,1600)
truth = torch.ones(batchsize,channel_num,256,1600)

dice = dice_channel_torch(probability, truth, 0.5)

print('Avg Dice score in this batch is {}'.format(dice))